

Tôi sẽ tách code thành các module riêng biệt:

1. `utils_convert_roles_for_api.py`:
```python:6_TuningWith2Prompting/utils_convert_roles_for_api.py
def convert_roles_for_api(messages, is_roleA_turn=True):
    """
    Chuyển đổi roleA/roleB thành user/assistant cho OpenAI API
    is_roleA_turn: True nếu đang là lượt của roleA, False nếu là lượt của roleB
    """
    converted_messages = []
    for msg in messages:
        if is_roleA_turn:
            if msg["role"] == "roleA":
                role = "assistant"
            else:
                role = "user"
        else:
            if msg["role"] == "roleB":
                role = "assistant"
            else:
                role = "user"
        
        converted_messages.append({
            "role": role,
            "content": msg["content"]
        })
    return converted_messages
```

2. `def_promptA.py`:
```python:6_TuningWith2Prompting/def_promptA.py
from openai import OpenAI
import time
from utils_convert_roles_for_api import convert_roles_for_api

def generate_roleA_response(client, roleA_prompt, message_history):
    """Generate response for roleA"""
    print("\n=== RoleA Turn ===")
    print("Original message history:")
    print(json.dumps(message_history, indent=2, ensure_ascii=False))
    
    api_messages = [{"role": "system", "content": roleA_prompt}]
    if message_history:
        converted_history = convert_roles_for_api(message_history, is_roleA_turn=True)
        api_messages.extend(converted_history)
        print("\nConverted history for RoleA:")
        print(json.dumps(api_messages, indent=2, ensure_ascii=False))
    
    start_time = time.time()
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=api_messages,
        temperature=0.3
    )
    end_time = time.time()
    
    return response.choices[0].message.content, end_time - start_time
```

3. `def_promptB.py`:
```python:6_TuningWith2Prompting/def_promptB.py
from openai import OpenAI
import time
from utils_convert_roles_for_api import convert_roles_for_api

def generate_roleB_response(client, roleB_prompt, message_history):
    """Generate response for roleB"""
    print("\n=== RoleB Turn ===")
    print("Original message history:")
    print(json.dumps(message_history, indent=2, ensure_ascii=False))
    
    api_messages = [{"role": "system", "content": roleB_prompt}]
    if message_history:
        converted_history = convert_roles_for_api(message_history, is_roleA_turn=False)
        api_messages.extend(converted_history)
        print("\nConverted history for RoleB:")
        print(json.dumps(api_messages, indent=2, ensure_ascii=False))
    
    start_time = time.time()
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=api_messages,
        temperature=0
    )
    end_time = time.time()
    
    return response.choices[0].message.content, end_time - start_time
```

4. `utils_export_conversations_to_excel.py`:
```python:6_TuningWith2Prompting/utils_export_conversations_to_excel.py
import pandas as pd

def prepare_export_data(all_messages):
    """Prepare data for export"""
    print("\n=== Preparing to Export Data ===")
    print(f"Total messages to export: {len(all_messages)}")
    print("First few rows of data:")
    for i, msg in enumerate(all_messages[:3]):
        print(f"Row {i+1}: {msg}")
    return all_messages

def validate_export_path(output_path):
    """Validate export path"""
    if not output_path.endswith('.xlsx'):
        output_path += '.xlsx'
    return output_path
```

5. `export_conversations_to_excel.py`:
```python:6_TuningWith2Prompting/export_conversations_to_excel.py
import pandas as pd
from utils_export_conversations_to_excel import prepare_export_data, validate_export_path

def export_conversations_to_excel(all_messages, output_path):
    """Export all conversations and response times to an Excel file."""
    try:
        # Prepare data
        data = prepare_export_data(all_messages)
        
        # Validate output path
        output_path = validate_export_path(output_path)
        
        # Create DataFrame and export
        df_export = pd.DataFrame(data, 
                               columns=['Role', 'Content', 'Response_Time', 
                                      'RoleA_Prompt', 'RoleB_Prompt'])
        df_export.to_excel(output_path, index=False)
        print(f"Conversations exported to {output_path}")
        
    except PermissionError:
        print("Error: Please close the Excel file before saving.")
    except Exception as e:
        print(f"Error during export: {str(e)}")
```

6. `main_v2.py`:
```python:6_TuningWith2Prompting/main_v2.py
import json
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv
import os
import argparse
from def_promptA import generate_roleA_response
from def_promptB import generate_roleB_response
from export_conversations_to_excel import export_conversations_to_excel

# Load environment variables
load_dotenv()
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

def simulate_conversation(row):
    """Simulate a conversation based on the row of data."""
    message_history = []
    response_times = []
    conversationTurnCount = 0

    # Extract and validate settings
    roleA_prompt = str(row['roleA_prompt']) if not pd.isna(row['roleA_prompt']) else ""
    roleB_prompt = str(row['roleB_prompt']) if not pd.isna(row['roleB_prompt']) else ""
    initialConversationHistory = row['initialConversationHistory']
    maxTurns = int(row['maxTurns']) if not pd.isna(row['maxTurns']) else 3

    # Process conversation history
    if not pd.isna(initialConversationHistory):
        try:
            history = json.loads(initialConversationHistory)
            message_history.extend(history)
        except json.JSONDecodeError as e:
            print(f"Error parsing conversation history: {e}")

    # Conversation loop
    while conversationTurnCount < maxTurns:
        try:
            # RoleA's turn
            roleA_message, roleA_time = generate_roleA_response(client, roleA_prompt, message_history)
            message_history.append({"role": "roleA", "content": roleA_message})
            response_times.append(roleA_time)

            # RoleB's turn
            roleB_message, roleB_time = generate_roleB_response(client, roleB_prompt, message_history)
            message_history.append({"role": "roleB", "content": roleB_message})
            response_times.append(roleB_time)

            conversationTurnCount += 1

        except Exception as e:
            print(f"Error during conversation: {str(e)}")
            break

    return message_history, response_times

def main(num_rows_to_process=None, input_file='2PromptingTuning.xlsx', output_file='result.xlsx'):
    try:
        # Load and process data
        df = pd.read_excel(input_file)
        num_rows = min(num_rows_to_process or len(df), len(df))
        
        all_messages = []
        for index, row in df.head(num_rows).iterrows():
            print(f"\n=== Processing Row {index + 1}/{num_rows} ===")
            
            message_history, response_times = simulate_conversation(row)
            
            # Prepare export data
            for i, msg in enumerate(message_history):
                all_messages.append([
                    msg['role'],
                    msg['content'],
                    response_times[i] if i < len(response_times) else 0,
                    row['roleA_prompt'],
                    row['roleB_prompt']
                ])
            all_messages.append(['Separator', '-------------------', 0, '', ''])
        
        # Export results
        export_conversations_to_excel(all_messages, output_file)
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description='Process conversations from Excel file')
    parser.add_argument('--rows', type=int, help='Number of rows to process')
    parser.add_argument('--input', type=str, default='2PromptingTuning.xlsx')
    parser.add_argument('--output', type=str, default='result.xlsx')
    
    args = parser.parse_args()
    main(args.rows, args.input, args.output)
```

Cấu trúc thư mục sẽ là:
```
6_TuningWith2Prompting/
├── utils_convert_roles_for_api.py
├── def_promptA.py
├── def_promptB.py
├── utils_export_conversations_to_excel.py
├── export_conversations_to_excel.py
└── main_v2.py
```
